In [200]:
import pandas as pd
import numpy as np
from collections import Counter
import re
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split,GridSearchCV
from math import sqrt
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from xgboost import XGBRegressor
import pickle
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold
from sklearn.exceptions import NotFittedError
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
# Comment this if the data visualisations doesn't work on your side
%matplotlib inline

plt.style.use('bmh')

In [201]:
train = pd.read_excel('C:/Users/pf416e/Desktop/ML_Hack/Participants_Data_Used_Cars/Data_Train.xlsx')
test = pd.read_excel('C:/Users/pf416e/Desktop/ML_Hack/Participants_Data_Used_Cars/Data_Test.xlsx')

In [202]:
train['Name']=train['Name'].str.strip()
train['Name']=train['Name'].str.upper()
test['Name']=test['Name'].str.strip()
test['Name']=test['Name'].str.upper()


train['Location']=train['Location'].str.strip()
train['Location']=train['Location'].str.upper()
test['Location']=test['Location'].str.strip()
test['Location']=test['Location'].str.upper()

train['Fuel_Type']=train['Fuel_Type'].str.strip()
train['Fuel_Type']=train['Fuel_Type'].str.upper()
test['Fuel_Type']=test['Fuel_Type'].str.strip()
test['Fuel_Type']=test['Fuel_Type'].str.upper()


train['Transmission']=train['Transmission'].str.strip()
train['Transmission']=train['Transmission'].str.upper()
test['Transmission']=test['Transmission'].str.strip()
test['Transmission']=test['Transmission'].str.upper()


train['Owner_Type']=train['Owner_Type'].str.strip()
train['Owner_Type']=train['Owner_Type'].str.upper()
test['Owner_Type']=test['Owner_Type'].str.strip()
test['Owner_Type']=test['Owner_Type'].str.upper()



train['Mileage']=train['Mileage'].str.strip()
train['Mileage']=train['Mileage'].str.upper()
test['Mileage']=test['Mileage'].str.strip()
test['Mileage']=test['Mileage'].str.upper()



train['Engine']=train['Engine'].str.strip()
train['Engine']=train['Engine'].str.upper()
test['Engine']=test['Engine'].str.strip()
test['Engine']=test['Engine'].str.upper()



train['Power']=train['Power'].str.strip()
train['Power']=train['Power'].str.upper()
test['Power']=test['Power'].str.strip()
test['Power']=test['Power'].str.upper()



train['New_Price']=train['New_Price'].str.strip()
train['New_Price']=train['New_Price'].str.upper()
test['New_Price']=test['New_Price'].str.strip()
test['New_Price']=test['New_Price'].str.upper()

In [203]:
#Outlier Treatment:

#train=train[(train['Kilometers_Driven']  > 999) & (train['Kilometers_Driven'] < 6500000)]
#train = train.reset_index(drop=True)

In [204]:
tg=train[['Price']]
del train['Price']

In [205]:
train.shape

(6019, 12)

In [206]:
all_data=pd.concat([train,test])
all_data = all_data.reset_index(drop=True)

In [207]:
#all_data['CR_fg']=np.where(all_data['New_Price'].str.contains('CR',na=False),1,0)
#all_data['LAKH_fg']=np.where(all_data['New_Price'].str.contains('LAKH',na=False),1,0)
#all_data['KMKG_fg']=np.where(all_data['Mileage'].str.contains('KM/KG',na=False),1,0)
#all_data['KMPL_fg']=np.where(all_data['Mileage'].str.contains('KMPL',na=False),1,0)
#all_data['NULL_fg']=np.where(all_data['Power'].str.contains('NULL',na=False),1,0)

In [208]:
all_data.shape

(7253, 12)

In [209]:
names1 = list(all_data['Name'])
for i in range(len(names1)) :
    try:
       names1[i] = names1[i].split(" ")[0].strip().upper()
    except :
       pass 

In [210]:
names2 = list(all_data['Name'])
for i in range(len(names2)) :
    try:
       names2[i] = names2[i].split(" ")[1].strip().upper()
    except :
       pass 

In [211]:
car_data = {}
car_data['Brand'] = names1
car_data['Sub_Brand'] = names2
car_data = pd.DataFrame(car_data)

In [212]:
all_data['Name']=all_data['Name'].str.upper()
all_data['Location']=all_data['Location'].str.upper()
all_data['Fuel_Type']=all_data['Fuel_Type'].str.upper()
all_data['Transmission']=all_data['Transmission'].str.upper()
all_data['Owner_Type']=all_data['Owner_Type'].str.upper()

In [213]:
#Cleaning Engine, Power and Mileage

all_data['Engine']=all_data['Engine'].str.replace('[A-Za-z]','')
all_data['Engine'] = all_data['Engine'].str.strip()
all_data['Engine'] = all_data['Engine'].apply(float)

all_data['Power']=all_data['Power'].str.replace('[A-Za-z]','')
all_data['Power'] = all_data['Power'].str.strip()
all_data.Power = all_data.Power.fillna('')


all_data['Mileage']=all_data['Mileage'].str.replace('[A-Za-z,/]','')
all_data['Mileage'] = all_data['Mileage'].str.strip()
all_data['Mileage'] = all_data['Mileage'].apply(float)

In [214]:
Power = list(all_data['Power'])

for i in range(len(Power)) :
    try:
       Power[i] = float(Power[i])
    except :
       Power[i] = np.nan
    

num_data = {}
num_data['Power'] = Power
num_data = pd.DataFrame(num_data)

In [215]:
del all_data['Power']
all_data=pd.concat([all_data,num_data,car_data], axis=1)

In [216]:
new = all_data['New_Price'].str.split(" ", n = 1, expand = True) 
all_data["Price"]= new[0] 
all_data['Price'] = all_data['Price'].apply(float)
all_data["Currency"]= new[1]

all_data['Cr']=np.where(all_data['Currency'].str.contains('Cr'),1,0)

In [217]:
all_data["Final_Price"]= np.where(all_data['Cr']==1, all_data["Price"]*100, all_data["Price"])

In [218]:
del all_data['New_Price']
del all_data['Price']
del all_data['Currency']
del all_data['Cr']

In [219]:
all_data['History']=2019 - all_data['Year']

In [220]:
all_data['Year'] = all_data['Year'].apply(str)

In [221]:
t1=all_data.isnull().sum()
t1=pd.DataFrame(t1)
t1.columns=['Values']
t1[t1.Values>0]

,Values
Mileage,2
Engine,46
Seats,53
Power,175
Final_Price,6247


In [222]:
impute_val=all_data.groupby(['Brand','Sub_Brand','Fuel_Type', 'Transmission'], as_index=False)[['Seats', 'Power','Engine','Mileage']].median()
impute_val.columns=['Brand','Sub_Brand','Fuel_Type', 'Transmission','Seats1','Power1','Engine1','Mileage1']

In [223]:
all_data_v2=pd.merge(all_data, impute_val, on=['Brand','Sub_Brand','Fuel_Type', 'Transmission'], how='left')

In [224]:
all_data_v2["Seats_v1"]= np.where(all_data_v2['Seats']> 0, all_data_v2['Seats'], all_data_v2['Seats1'])
all_data_v2["Power_v1"]= np.where(all_data_v2['Power']> 0, all_data_v2['Power'], all_data_v2['Power1'])
all_data_v2["Engine_v1"]= np.where(all_data_v2['Engine']> 0, all_data_v2['Engine'], all_data_v2['Engine1'])
all_data_v2["Mileage_v1"]= np.where(all_data_v2['Mileage']> 0, all_data_v2['Mileage'], all_data_v2['Mileage1'])

In [225]:
del all_data_v2['Seats']
del all_data_v2['Seats1']

del all_data_v2['Power']
del all_data_v2['Power1']

del all_data_v2['Mileage']
del all_data_v2['Mileage1']

del all_data_v2['Engine']
del all_data_v2['Engine1']

In [226]:
all_data_v2['Seats_v1']=all_data_v2['Seats_v1'].fillna(all_data_v2['Seats_v1'].median())
all_data_v2['Power_v1']=all_data_v2['Power_v1'].fillna(all_data_v2['Power_v1'].median())
all_data_v2['Engine_v1']=all_data_v2['Engine_v1'].fillna(all_data_v2['Engine_v1'].median())
all_data_v2['Mileage_v1']=all_data_v2['Mileage_v1'].fillna(all_data_v2['Mileage_v1'].median())


all_data_v2['Final_Price']=all_data_v2['Final_Price'].fillna(-999)

In [227]:
cat_cols=all_data_v2.columns[all_data_v2.dtypes=='object'].tolist()
print(cat_cols)

num_cols=all_data_v2.columns[all_data_v2.dtypes!='object'].tolist()
print(num_cols)

['Name', 'Location', 'Year', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Brand', 'Sub_Brand']
['Kilometers_Driven', 'Final_Price', 'History', 'Seats_v1', 'Power_v1', 'Engine_v1', 'Mileage_v1']


In [228]:
# Additional Removal Part:
train1_dup1=all_data_v2[:6019]
test1_dup1=all_data_v2[6019:7253]

train1_dup2=pd.concat([train1_dup1,tg], axis=1)

train1_dup2['flag']=train1_dup2['Fuel_Type']+train1_dup2['Location']+train1_dup2['Name']+train1_dup2['Owner_Type']+train1_dup2['Transmission']+train1_dup2['Year']+train1_dup2['Brand']+train1_dup2['Sub_Brand']+train1_dup2['Kilometers_Driven'].astype(str)+train1_dup2['Final_Price'].astype(str)+ train1_dup2['History'].astype(str)+train1_dup2['Seats_v1'].astype(str)+train1_dup2['Power_v1'].astype(str)+train1_dup2['Engine_v1'].astype(str)+train1_dup2['Mileage_v1'].astype(str)

dup1=train1_dup2[train1_dup2.flag.duplicated()]
dup2=dup1.drop_duplicates(subset ="flag",keep='first')
dup2=dup2[['flag']]

dup_inner=pd.merge(dup2, train1_dup2, on='flag', how='inner')

dup_inner2=dup_inner.groupby('flag', as_index=False).agg({"Price": "mean"})
dup_inner2.columns = ['flag', 'Price1']

dup_inner3=pd.merge(dup_inner,dup_inner2, on='flag', how='inner')

dup_inner4=dup_inner3.drop_duplicates(subset ="flag",keep='first') 

del dup_inner4['flag']
del dup_inner4['Price']

dup_inner4 = dup_inner4.rename(columns = {"Price1": "Price"})

train1_dup3=train1_dup2.drop_duplicates(subset ="flag",keep = False)

del train1_dup3['flag']

train1_dup4=pd.concat([train1_dup3,dup_inner4])
train1_dup4 = train1_dup4.reset_index(drop=True)

In [229]:
tg=train1_dup4[['Price']]
tg["Price1"] = np.log1p(tg["Price"].astype(np.float64))

target=tg.Price1#Change
del train1_dup4['Price']

all_data_v2=pd.concat([train1_dup4,test1_dup1])
all_data_v2 = all_data_v2.reset_index(drop=True)

C:\Users\pf416e\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [230]:
all_data_v2.shape


(7235, 15)

In [231]:
all_data_v2.describe()

,Kilometers_Driven,Final_Price,History,Seats_v1,Power_v1,Engine_v1,Mileage_v1
count,7.235000e+03,7235.000000,7235.000000,7235.000000,7235.000000,7235.000000,7235.000000
mean,5.873222e+04,-857.326511,5.633725,5.278784,112.134200,1615.251831,18.294951
std,8.451807e+04,352.633886,3.255190,0.807305,53.309144,594.226306,4.204655
min,1.710000e+02,-999.000000,0.000000,2.000000,34.200000,72.000000,0.000000
25%,3.400000e+04,-999.000000,3.000000,5.000000,74.000000,1198.000000,15.290000
50%,5.350000e+04,-999.000000,5.000000,5.000000,92.850000,1493.000000,18.160000
75%,7.300000e+04,-999.000000,8.000000,5.000000,138.030000,1968.000000,21.100000
max,6.500000e+06,99.920000,23.000000,10.000000,616.000000,5998.000000,33.540000


In [232]:
class Encoding(BaseEstimator):
    categorical_columns = None
    return_df = False
    random_state = 30
    threshold = 50

    def __init__(self):
        pass

    def convert_input(self, X):
        if not isinstance(X, pd.DataFrame):
            if isinstance(X, list):
                X = pd.DataFrame(np.array(X))
            elif isinstance(X, (np.generic, np.ndarray, pd.Series)):
                X = pd.DataFrame(X)
            else:
                raise ValueError('Unexpected input type: %s' % (str(type(X))))
            X = X.apply(lambda x: pd.to_numeric(x, errors='ignore'))
        x = X.copy(deep = True)
        return x

    def get_categorical_columns(self, X):
        return X.select_dtypes(include=['object', 'category']).columns.tolist()

    def get_numerical_columns(self,X):
        temp_x=X[X.columns[X.nunique()<=self.threshold]]
        col_names=temp_x.columns[temp_x.dtypes!='object']
        return col_names

    def apply_encoding(self, X_in, encoding_dict):
        X = self.convert_input(X_in)
        for col in self.categorical_columns:
            if col in encoding_dict:
                freq_dict = encoding_dict[col]
                X[col] = X[col].apply(lambda x: freq_dict[x] if x  in freq_dict else np.nan)
        return X

    def create_encoding_dict(self, X, y):
        return {}

    def fit(self, X, y=None):
        if X is None:
            raise ValueError("Input array is required to call fit method!")
        X = self.convert_input(X)
        self.encoding_dict = self.create_encoding_dict(X, y)
        return self

    def transform(self, X):
        df = self.apply_encoding(X, self.encoding_dict)
        if self.return_df:
            return df
        else:
            return df.values

    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)

    def inverse_transform(self, X):
        X = self.convert_input(X)
        for col in self.categorical_columns:
            freq_dict = self.encoding_dict[col]
            for key, val in freq_dict.iteritems():
                X.loc[X[col] == val, col] = key
        if self.return_df:
            return X
        else:
            return X.values

In [233]:
class FreqeuncyEncoding(Encoding):
    '''
    class to perform FreqeuncyEncoding on Categorical Variables
    Initialization Variabes:
    categorical_columns: list of categorical columns from the dataframe
    or list of indexes of caategorical columns for numpy ndarray
    return_df: boolean
        if True: returns pandas dataframe on transformation
        else: return numpy ndarray
    '''
    def __init__(self, categorical_columns = None, return_df = False):
        self.categorical_columns = categorical_columns
        self.return_df = return_df

    def create_encoding_dict(self, X, y):
        encoding_dict = {}
        if self.categorical_columns is None:
            self.categorical_columns = self.get_categorical_columns(X)
        for col in self.categorical_columns:
            encoding_dict.update({col: X[col].value_counts(normalize = True).to_dict()})
        return encoding_dict


fe=FreqeuncyEncoding(categorical_columns=cat_cols,return_df=True)
all_data_v3=fe.fit_transform(all_data_v2)

In [234]:
titles = list(all_data_v2['Location'])

maxim = 1
for i in titles :
    if len(i.split(',')) > maxim:
         maxim = len(i.split(','))
         
print("\n\nMaximum Titles in a Cell : ", maxim)    

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())

print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  1


Number of Unique Titles :  11


Unique Titles:
 ['MUMBAI' 'PUNE' 'CHENNAI' 'COIMBATORE' 'HYDERABAD' 'JAIPUR' 'KOCHI'
 'KOLKATA' 'DELHI' 'BANGALORE' 'AHMEDABAD']


In [235]:
for i in range(11):
    ttl=all_titles[i]
    ttl1=ttl+'1'
    all_data_v2[ttl1] = all_data_v2['Location'].str.contains(ttl)
    all_data_v2[ttl1] = all_data_v2[ttl1].map({True: 1, False: 0}) 
    
del all_data_v2['Location']

In [236]:
titles = list(all_data_v2['Fuel_Type'])

maxim = 1
for i in titles :
    if len(i.split(',')) > maxim:
         maxim = len(i.split(','))
         
print("\n\nMaximum Titles in a Cell : ", maxim)    

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())

print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  1


Number of Unique Titles :  5


Unique Titles:
 ['CNG' 'DIESEL' 'PETROL' 'LPG' 'ELECTRIC']


In [237]:
for i in range(5):
    ttl=all_titles[i]
    ttl2=ttl+'2'
    all_data_v2[ttl2] = all_data_v2['Fuel_Type'].str.contains(ttl)
    all_data_v2[ttl2] = all_data_v2[ttl2].map({True: 1, False: 0})
    
del all_data_v2['Fuel_Type']

In [238]:
titles = list(all_data_v2['Transmission'])

maxim = 1
for i in titles :
    if len(i.split(',')) > maxim:
         maxim = len(i.split(','))
         
print("\n\nMaximum Titles in a Cell : ", maxim)    

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())

print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  1


Number of Unique Titles :  2


Unique Titles:
 ['MANUAL' 'AUTOMATIC']


In [239]:
for i in range(2):
    ttl=all_titles[i]
    ttl2=ttl+'3'
    all_data_v2[ttl2] = all_data_v2['Transmission'].str.contains(ttl)
    all_data_v2[ttl2] = all_data_v2[ttl2].map({True: 1, False: 0})
    
del all_data_v2['Transmission']

In [240]:
titles = list(all_data_v2['Owner_Type'])

maxim = 1
for i in titles :
    if len(i.split(',')) > maxim:
         maxim = len(i.split(','))
         
print("\n\nMaximum Titles in a Cell : ", maxim)    

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())

print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  1


Number of Unique Titles :  4


Unique Titles:
 ['FIRST' 'SECOND' 'FOURTH & ABOVE' 'THIRD']


In [241]:
for i in range(4):
    ttl=all_titles[i]
    ttl2=ttl+'4'
    all_data_v2[ttl2] = all_data_v2['Owner_Type'].str.contains(ttl)
    all_data_v2[ttl2] = all_data_v2[ttl2].map({True: 1, False: 0})
    
del all_data_v2['Owner_Type']

In [242]:
titles = list(all_data_v2['Year'])

maxim = 1
for i in titles :
    if len(i.split(',')) > maxim:
         maxim = len(i.split(','))
         
print("\n\nMaximum Titles in a Cell : ", maxim)    

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())

print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  1


Number of Unique Titles :  23


Unique Titles:
 ['2010' '2015' '2011' '2012' '2013' '2016' '2018' '2014' '2017' '2007'
 '2009' '2008' '2019' '2006' '2005' '2004' '2002' '2000' '2003' '1999'
 '2001' '1998' '1996']


In [243]:
for i in range(23):
    ttl=all_titles[i]
    ttl2=ttl+'_5'
    all_data_v2[ttl2] = all_data_v2['Year'].str.contains(ttl)
    all_data_v2[ttl2] = all_data_v2[ttl2].map({True: 1, False: 0})
    
del all_data_v2['Year']

In [244]:
titles = list(all_data_v2['Brand'])

maxim = 1
for i in titles :
    if len(i.split(',')) > maxim:
         maxim = len(i.split(','))
         
print("\n\nMaximum Titles in a Cell : ", maxim)    

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())

print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  1


Number of Unique Titles :  32


Unique Titles:
 ['MARUTI' 'HYUNDAI' 'HONDA' 'AUDI' 'NISSAN' 'TOYOTA' 'VOLKSWAGEN' 'TATA'
 'LAND' 'MITSUBISHI' 'RENAULT' 'MERCEDES-BENZ' 'BMW' 'MAHINDRA' 'FORD'
 'PORSCHE' 'DATSUN' 'JAGUAR' 'VOLVO' 'CHEVROLET' 'SKODA' 'MINI' 'FIAT'
 'JEEP' 'SMART' 'AMBASSADOR' 'ISUZU' 'FORCE' 'BENTLEY' 'LAMBORGHINI'
 'HINDUSTAN' 'OPELCORSA']


In [245]:
for i in range(32):
    ttl=all_titles[i]
    ttl2=ttl+'_6'
    all_data_v2[ttl2] = all_data_v2['Brand'].str.contains(ttl)
    all_data_v2[ttl2] = all_data_v2[ttl2].map({True: 1, False: 0})
    
del all_data_v2['Brand']

In [246]:
titles = list(all_data_v2['Sub_Brand'])

maxim = 1
for i in titles :
    if len(i.split(',')) > maxim:
         maxim = len(i.split(','))
         
print("\n\nMaximum Titles in a Cell : ", maxim)    

all_titles = []

for i in titles :
    if len(i.split(',')) == 1:
         all_titles.append(i.split(',')[0].strip().upper())
    else :
        for it in range(len(i.split(','))):
            all_titles.append(i.split(',')[it].strip().upper())

print("\n\nNumber of Unique Titles : ", len(pd.Series(all_titles).unique()))
print("\n\nUnique Titles:\n", pd.Series(all_titles).unique())

all_titles = list(pd.Series(all_titles).unique())



Maximum Titles in a Cell :  1


Number of Unique Titles :  218


Unique Titles:
 ['WAGON' 'CRETA' 'JAZZ' 'ERTIGA' 'A4' 'EON' 'MICRA' 'INNOVA' 'VENTO'
 'INDICA' 'CIAZ' 'CITY' 'SWIFT' 'ROVER' 'PAJERO' 'AMAZE' 'DUSTER' 'NEW'
 '3' 'S' 'A6' 'I20' 'ALTO' 'WRV' 'COROLLA' 'SSANGYONG' 'VITARA' 'KUV'
 'M-CLASS' 'POLO' 'NANO' 'ELANTRA' 'XCENT' 'THAR' 'GRAND' 'KWID' 'I10'
 'X-TRAIL' 'ZEN' 'FIGO' 'C-CLASS' 'CAYENNE' 'XUV500' 'TERRANO' 'BRIO'
 'FIESTA' 'SANTRO' 'ZEST' 'RITZ' '5' 'FORTUNER' 'ECOSPORT' 'VERNA' 'GO'
 'OMNI' 'ETIOS' 'XF' 'EECO' 'CIVIC' 'V40' 'B' 'SCORPIO' 'CR-V' 'SLC' '1'
 'BEAT' 'RAPID' 'RS5' 'SUPERB' 'X5' 'GLC' 'COUNTRYMAN' 'OPTRA' 'LODGY'
 'E-CLASS' 'BALENO' 'LAURA' 'NUVOSPORT' 'FABIA' 'INDIGO' 'Q3' 'OCTAVIA'
 'A8' 'VERITO' 'COOPER' 'SANTA' 'X1' 'ACCENT' 'TUCSON' 'GLE' 'A-STAR'
 'GRANDE' 'X3' 'Q7' 'JETTA' 'GLA' 'CELERIO' 'SUMO' 'ACCORD' '6' 'MANZA'
 'SPARK' 'CLUBMAN' 'TEANA' '800' 'BRV' 'XE' 'XENON' 'A3' 'GL-CLASS' 'BR-V'
 'S80' 'CAPTUR' 'ENJOY' 'BOLERO' 'CEDIA' 'S-CROSS' 'YETI' 'E

In [247]:
for i in range(218):
    ttl=all_titles[i]
    ttl2=ttl+'_6'
    all_data_v2[ttl2] = all_data_v2['Sub_Brand'].str.contains(ttl)
    all_data_v2[ttl2] = all_data_v2[ttl2].map({True: 1, False: 0})
    
del all_data_v2['Sub_Brand']

In [248]:
all_data_v2.columns

Index(['Name', 'Kilometers_Driven', 'Final_Price', 'History', 'Seats_v1',
       'Power_v1', 'Engine_v1', 'Mileage_v1', 'MUMBAI1', 'PUNE1',
       ...
       'WR-V_6', 'CONTINENTAL_6', 'GALLARDO_6', 'F_6', 'MOTORS_6', 'FLYING_6',
       'MU_6', '370Z_6', 'ABARTH_6', '1.4GSI_6'],
      dtype='object', length=300)

In [249]:
all_data_v3.columns

Index(['Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type',
       'Transmission', 'Owner_Type', 'Brand', 'Sub_Brand', 'Final_Price',
       'History', 'Seats_v1', 'Power_v1', 'Engine_v1', 'Mileage_v1'],
      dtype='object')

In [250]:
all_data_v4=all_data_v3[['Name', 'Location', 'Year', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Brand', 'Sub_Brand']]

del all_data_v2['Name']
#del all_data_v2['Sub_Brand']

all_data_v5=pd.concat([all_data_v2,all_data_v4], axis=1)

In [251]:
train1=all_data_v5[:6001]
test1=all_data_v5[6001:7235]

In [252]:
grb=GradientBoostingRegressor(learning_rate=0.1,
                              n_estimators=500, 
                              max_depth=10,
                              subsample=0.8,
                              verbose=False,
                              random_state=1234)

grb.fit(train1, target)

t1=grb.feature_importances_
t1=pd.DataFrame(t1)

t2=train1.columns
t2=pd.DataFrame(t2)

t1.columns=['Importance']
t2.columns=['Variable']

t3=pd.concat([t2,t1],axis=1)
t4=t3[t3['Importance'] > 0]

#t4=t4.sort_values(by=['Importance'],ascending=False)
#t4["cumpercentage"] = t4["Importance"].cumsum()/t4["Importance"].sum()*100

my_cols = list(t4.Variable)
all_data_new_v1 = all_data_v5[my_cols]
all_data_new_v1.head()

train1=all_data_new_v1[:6001]
test1=all_data_new_v1[6001:7235]

In [253]:
#Variable Selection:

#t4=t4[t4['cumpercentage'] < 99]

#my_cols = list(t4.Variable)
#all_data_new_v1 = all_data_v5[my_cols]
#all_data_new_v1.head()

#train1=all_data_new_v1[:6001]
#test1=all_data_new_v1[6001:7235]

In [254]:
from sklearn.metrics import mean_squared_log_error
import math
def rmsle(real, predicted):
    real=np.expm1(real)
    predicted=np.expm1(predicted)
    return np.sqrt(mean_squared_log_error(real,predicted))
    
def rmsle_lgb(labels, preds):
    return 'rmsle', rmsle(preds,labels)

In [255]:
kf = StratifiedKFold(n_splits=15, shuffle=True, random_state=1234) #15
predicts = []
predicts_train = []
errgb=[]
for train_index, test_index in kf.split(train1, target):
    print("###")
    X_train, X_val = train1.iloc[train_index], train1.iloc[test_index]
    y_train, y_val = target.iloc[train_index], target.iloc[test_index]
    gb_params = {'learning_rate' : 0.2, #Done-0.2
                  'n_estimators' : 1000,#Done-900/1000
                  'subsample' : 1, #1 
                  'max_depth' : 4, #Done-4
                  'random_state' : 1234, 
                  'alpha' : 0.9, 
                  'verbose' : 100,
                  'max_features':0.3 #Done-0.3 
                  }
    clf=GradientBoostingRegressor(**gb_params)
    clf.fit(X_train, y_train)
    
    ypred = clf.predict(X_train)
    ypred_val = clf.predict(X_val)
    
    pr=clf.predict(X_val)
    print("errgb: ",rmsle_lgb(y_val.values,pr)[1])    
    errgb.append(rmsle_lgb(y_val.values,pr)[1])
    
    print(sqrt(mean_squared_error(y_train, ypred)))
    print(sqrt(mean_squared_error(y_val, ypred_val)))
    
    predicts.append(clf.predict(test1))
    predicts_train.append(clf.predict(train1))

C:\Users\pf416e\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=15.
  % (min_groups, self.n_splits)), Warning)


###
      Iter       Train Loss   Remaining Time 
         1           0.4103           20.49s
         2           0.3099           22.07s
         3           0.2334           23.34s
         4           0.1851           21.74s
         5           0.1498           24.05s
         6           0.1223           23.93s
         7           0.1008           23.51s
         8           0.0860           23.21s
         9           0.0753           23.16s
        10           0.0678           22.74s
        11           0.0612           23.31s
        12           0.0561           23.99s
        13           0.0520           23.70s
        14           0.0492           23.80s
        15           0.0467           23.26s
        16           0.0437           23.50s
        17           0.0415           23.35s
        18           0.0394           23.64s
        19           0.0382           23.89s
        20           0.0369           23.51s
        21           0.0359           23.13s
     

       183           0.0119           14.37s
       184           0.0118           14.33s
       185           0.0118           14.30s
       186           0.0118           14.25s
       187           0.0117           14.21s
       188           0.0117           14.16s
       189           0.0117           14.12s
       190           0.0116           14.07s
       191           0.0116           14.03s
       192           0.0116           14.01s
       193           0.0115           13.97s
       194           0.0115           13.92s
       195           0.0115           13.87s
       196           0.0115           13.84s
       197           0.0114           13.75s
       198           0.0113           13.83s
       199           0.0113           13.79s
       200           0.0112           13.79s
       201           0.0112           13.77s
       202           0.0112           13.75s
       203           0.0111           13.71s
       204           0.0111           13.64s
       205

       374           0.0074           10.51s
       375           0.0074           10.49s
       376           0.0074           10.46s
       377           0.0073           10.45s
       378           0.0073           10.43s
       379           0.0073           10.41s
       380           0.0073           10.41s
       381           0.0073           10.39s
       382           0.0073           10.38s
       383           0.0072           10.37s
       384           0.0072           10.37s
       385           0.0072           10.35s
       386           0.0072           10.35s
       387           0.0072           10.31s
       388           0.0071           10.32s
       389           0.0071           10.30s
       390           0.0071           10.28s
       391           0.0071           10.27s
       392           0.0071           10.25s
       393           0.0071           10.23s
       394           0.0070           10.22s
       395           0.0070           10.20s
       396

       557           0.0053            7.79s
       558           0.0052            7.78s
       559           0.0052            7.76s
       560           0.0052            7.74s
       561           0.0052            7.72s
       562           0.0052            7.70s
       563           0.0052            7.69s
       564           0.0052            7.68s
       565           0.0052            7.67s
       566           0.0052            7.65s
       567           0.0052            7.63s
       568           0.0052            7.62s
       569           0.0051            7.61s
       570           0.0051            7.58s
       571           0.0051            7.58s
       572           0.0051            7.57s
       573           0.0051            7.56s
       574           0.0051            7.55s
       575           0.0051            7.53s
       576           0.0050            7.53s
       577           0.0050            7.51s
       578           0.0050            7.49s
       579

...

        94           0.0167           16.93s
        95           0.0166           16.91s
        96           0.0165           16.72s
        97           0.0164           16.68s
        98           0.0164           16.65s
        99           0.0163           16.60s
       100           0.0163           16.43s
       101           0.0163           16.39s
       102           0.0162           16.36s
       103           0.0160           16.32s
       104           0.0159           16.29s
       105           0.0159           16.27s
       106           0.0158           16.24s
       107           0.0158           16.21s
       108           0.0157           16.18s
       109           0.0156           16.16s
       110           0.0155           16.24s
       111           0.0153           16.23s
       112           0.0153           16.17s
       113           0.0153           16.03s
       114           0.0152           16.01s
       115           0.0151           15.97s
       116

       277           0.0091           11.94s
       278           0.0091           11.92s
       279           0.0090           11.91s
       280           0.0090           11.93s
       281           0.0090           11.91s
       282           0.0089           11.90s
       283           0.0089           11.89s
       284           0.0089           11.86s
       285           0.0089           11.83s
       286           0.0088           11.84s
       287           0.0088           11.84s
       288           0.0088           11.80s
       289           0.0087           11.80s
       290           0.0087           11.79s
       291           0.0087           11.79s
       292           0.0087           11.75s
       293           0.0086           11.76s
       294           0.0086           11.74s
       295           0.0086           11.70s
       296           0.0086           11.70s
       297           0.0086           11.66s
       298           0.0086           11.64s
       299

       464           0.0061            8.81s
       465           0.0061            8.79s
       466           0.0061            8.79s
       467           0.0061            8.79s
       468           0.0060            8.78s
       469           0.0060            8.76s
       470           0.0060            8.75s
       471           0.0060            8.73s
       472           0.0060            8.71s
       473           0.0060            8.70s
       474           0.0060            8.68s
       475           0.0060            8.66s
       476           0.0059            8.65s
       477           0.0059            8.63s
       478           0.0059            8.61s
       479           0.0059            8.59s
       480           0.0059            8.57s
       481           0.0059            8.54s
       482           0.0059            8.51s
       483           0.0059            8.50s
       484           0.0059            8.50s
       485           0.0059            8.48s
       486

       652           0.0043            5.94s
       653           0.0043            5.93s
       654           0.0043            5.91s
       655           0.0043            5.89s
       656           0.0043            5.88s
       657           0.0042            5.87s
       658           0.0042            5.85s
       659           0.0042            5.83s
       660           0.0042            5.81s
       661           0.0042            5.80s
       662           0.0042            5.77s
       663           0.0042            5.76s
       664           0.0042            5.75s
       665           0.0042            5.73s
       666           0.0042            5.71s
       667           0.0042            5.69s
       668           0.0042            5.68s
       669           0.0042            5.66s
       670           0.0042            5.65s
       671           0.0042            5.63s
       672           0.0042            5.62s
       673           0.0041            5.60s
       674

       836           0.0032            2.88s
       837           0.0032            2.86s
       838           0.0032            2.85s
       839           0.0032            2.83s
       840           0.0032            2.81s
       841           0.0032            2.80s
       842           0.0032            2.78s
       843           0.0032            2.76s
       844           0.0032            2.75s
       845           0.0032            2.73s
       846           0.0032            2.71s
       847           0.0032            2.69s
       848           0.0032            2.67s
       849           0.0032            2.65s
       850           0.0032            2.64s
       851           0.0032            2.62s
       852           0.0031            2.60s
       853           0.0031            2.58s
       854           0.0031            2.56s
       855           0.0031            2.55s
       856           0.0031            2.53s
       857           0.0031            2.51s
       858

In [256]:
np.mean(errgb)

0.13864342969059826

In [257]:
s=pd.DataFrame({'Price':np.expm1(np.mean(predicts,0))})
s.to_excel('gb_impute_v3.xlsx', index=False)#0.9463

In [258]:
s=pd.DataFrame({'Price':np.expm1(np.min(predicts,0))}) #max, mean, min
s.to_excel('gb_impute_v3_min.xlsx', index=False)#0.9274

In [259]:
s=pd.DataFrame({'Price':np.expm1(np.max(predicts,0))}) #max, mean, min
s.to_excel('gb_impute_v3_max.xlsx', index=False)#0.9308

In [260]:
s=pd.DataFrame({'Price':np.expm1(np.median(predicts,0))}) #max, mean, min
s.to_excel('gb_impute_v3_median.xlsx', index=False)#0.9465

In [261]:
# For Stacking:

s=pd.DataFrame({'gb_Price':(np.median(predicts_train,0))}) #max, mean, min
s.to_excel('gb_train.xlsx', index=False)

s=pd.DataFrame({'gb_Price':(np.median(predicts,0))}) #max, mean, min
s.to_excel('gb_test.xlsx', index=False)

In [262]:
#Subsample:
#0.5:0.9450
#0.9:0.9454
#0.6:0.9461
#0.8:0.9457
#0.4:0.9455
#0.7:0.9455
#0.3:0.9448
#1:0.9465


In [263]:
#Max_features
#1:0.9444
#0.9:0.9441
#0.8:0.9447
#0.7:0.9444
#0.6:0.9456
#0.5:0.9451
#0.4:0.9456
#0.35:0.9451
#0.32:0.9448
#0.3:0.9465
#0.28:0.9448
#0.25:0.9450
#0.2:0.9446
#0.1:0.9455
#0.15:0.9455


#0.4:0.9439
#0.5:0.9435
#0.35:0.9439
#0.3:0.9447
#0.25:0.9443
#0.2:0.9444

In [264]:
#Depth:
#0.3,5:0.9447
#0.3,6:0.9429
#0.3,4:0.9452
#0.3,3:0.9446

In [265]:
#No. of Estimator:
#0.3,4,600:0.9452
#0.3,4,700:0.9446
#0.3,4,500:0.9449
#0.3,4,800:0.9455
#0.3,4,400:0.9444
#0.3,4,900:0.9455
#0.3,4,1000:0.9454
#0.3,4,1100:0.9454
#0.3,4,1200:0.9454

In [266]:
#Learning Rate:
#0.3,4,900,0.2:0.9455
#0.3,4,900,0.05:0.9414
#0.3,4,900,0.3:0.9449
#0.3,4,900,0.1:0.9441
#0.3,4,800,0.3:0.9449
#0.3,4,800,0.1:0.9438

In [267]:
#CV Fold:
#10:0.9455
#25:0.9458
#5:0.9444
#15:0.9463
#30:0.9458
#20:0.9455
#40:0.9455